In [1]:
!nvidia-smi

Wed Oct 11 11:09:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 460.27.04    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 30%   35C    P8     4W / 250W |   5888MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 30%   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

## 데이터 불러오기

In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train/255.0
x_test = x_test/255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## 사전학습된 모델 불러오기

In [4]:
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
pretrained_model = MobileNetV2(weights = 'imagenet', # 사전학습한 데이터
                    include_top = False,
                    input_shape = (96,96,3) # 사진 한장의 크기
                    )

2023-10-11 11:09:30.501303: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


In [5]:
for layer in pretrained_model.layers:
    layer.trainable = True # 사전에 학습되어있는 Layer의 미세조정을 허용하겠다.

## 모델에 추가

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

model = Sequential() # 빈그릇 만들기
model.add(pretrained_model) # 사전학습된 모델추가
model.add(GlobalAveragePooling2D()) # 미세조정 위한 마지막 단 추가
model.add(Dense(160,activation='relu'))  # 미세조정 위한 마지막 단 추가
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))  # 미세조정 위한 마지막 단 추가

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 160)               204960    
                                                                 
 dropout (Dropout)           (None, 160)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1610      
                                                                 
Total params: 2,464,554
Trainable params: 2,430,442
Non-

## 데이터 크기 변경

In [8]:
import tensorflow as tf
x_train = tf.image.resize(x_train,[96,96])
x_test = tf.image.resize(x_test,[96,96])

## 학습

In [9]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
history= model.fit(x_train,y_train,
                   batch_size=64,
                   epochs=10,
                   validation_data=(x_test,y_test))

Epoch 1/10


2023-10-11 11:09:45.738719: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.07GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-10-11 11:09:45.738800: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.07GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


782/782 [==============================] - 69s 79ms/step - loss: 0.6282 - accuracy: 0.8060 - val_loss: 6.9153 - val_accuracy: 0.2233
Epoch 2/10
782/782 [==============================] - 60s 76ms/step - loss: 0.3983 - accuracy: 0.8745 - val_loss: 3.1308 - val_accuracy: 0.4302
Epoch 3/10
782/782 [==============================] - 60s 77ms/step - loss: 0.3288 - accuracy: 0.8973 - val_loss: 1.3507 - val_accuracy: 0.6702
Epoch 4/10
782/782 [==============================] - 59s 75ms/step - loss: 0.2859 - accuracy: 0.9082 - val_loss: 1.7915 - val_accuracy: 0.5880
Epoch 5/10
782/782 [==============================] - 59s 75ms/step - loss: 0.2575 - accuracy: 0.9196 - val_loss: 3.3479 - val_accuracy: 0.5056
Epoch 6/10
782/782 [==============================] - 59s 76ms/step - loss: 0.2221 - accuracy: 0.9275 - val_loss: 2.4904 - val_accuracy: 0.6040
Epoch 7/10
782/782 [==============================] - 59s 75ms/step - loss: 0.1965 - accuracy: 0.9373 - val_loss: 1.3193 - val_accuracy: 0.7203
Epo